In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,BatchNorm2d
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [116]:
## change ##
batch_size = 16
clip = .25
name = 'spec_3d_phase_testY'
testY = pd.read_csv('../Submission/gbm0.csv')
pList = [0.9,0.1]

In [81]:
def transformFun(x):
    _,_,s1 = spectrogram(x,nperseg=256,noverlap=256//4,mode='phase')
    _,_,s2 = spectrogram(x,nperseg=256,noverlap=256//4)
    return np.stack([np.log(s2+1e-6),s1],0)

Prepare data

In [117]:
train_gen = SequenceGenTransform(train,transformFun)
temp_gen = SequenceGenTestY(testY,transformFun)
train_gen = SequenceCombine([train_gen,temp_gen],pList)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)


val_gen = SequenceGenTransform(train,transformFun,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [118]:
convNets = Sequential(BatchNorm2d(2),
                      ConvBatchLeaky(2,4,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(4,8,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(8,16,(5,5),stride=(2,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(16,32,(3,3),stride=(2,2)),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.view(-1,32*6,192)),
                      GRU_NCL(192,192,2,batch_first=True,dropout=0.5,bidirectional=True,returnH=True))

In [119]:
linear = Sequential(Dropout(0.5),Linear(192,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [120]:
# train from scratch 
# model = CNN_RNN2seq_transpose(convNets,linear).to('cuda:0')

In [121]:
# fine tune
model = CNN_RNN2seq_transpose(convNets,linear)
model.load_state_dict(torch.load('../Model/spec_3d_phase.pt'))
model = model.to('cuda:0')

In [122]:
opt = Adam(trainable_parameter(model),lr=2e-4)

In [123]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [124]:
# fine-tune on spec_3d_rnn_phase
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.052517512956491, val_loss:2.349804162979126
epoch:1, train_loss:2.0199566214130473, val_loss:2.370410203933716
epoch:2, train_loss:2.043987082174191, val_loss:2.345155715942383
epoch:3, train_loss:2.019066894856783, val_loss:2.3506112098693848
epoch:4, train_loss:2.0150464543929467, val_loss:2.3627336025238037
Training completed in 104.18790936470032s


In [88]:
# train from scratch 
model = fit(15, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.1456280344953904, val_loss:3.328080654144287
epoch:1, train_loss:2.6442493395163464, val_loss:2.6316452026367188
epoch:2, train_loss:2.1626721918582916, val_loss:2.7248167991638184
epoch:3, train_loss:2.151882865681098, val_loss:2.6437723636627197
epoch:4, train_loss:2.1047244100616527, val_loss:2.4694244861602783
epoch:5, train_loss:2.0898225098275223, val_loss:2.6106884479522705
epoch:6, train_loss:2.067967221713983, val_loss:2.6209089756011963
epoch:7, train_loss:2.0731137515260625, val_loss:2.6602070331573486
epoch:8, train_loss:2.051861607684539, val_loss:2.5590362548828125
epoch:9, train_loss:2.0509912087940254, val_loss:2.5647647380828857
epoch:10, train_loss:2.0512946935800405, val_loss:2.4581918716430664
epoch:11, train_loss:2.0564027978823733, val_loss:2.5388691425323486
epoch:12, train_loss:2.0372342524620204, val_loss:2.326343297958374
epoch:13, train_loss:2.0451640773278017, val_loss:2.414299964904785
epoch:14, train_loss:2.035420723259449, val_loss:2

Save Model and make Submission

In [3]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [34]:
test_gen = SequenceGenTransformTest(submission.seg_id.tolist(),transformFun)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [35]:
submission.iloc[:,1] = predict(model,test_gen)

In [36]:
submission.to_csv('../Submission/'+name+'.csv',index=False)

In [39]:
torch.save(model.state_dict(), '../Model/'+name+'.pt') 